## SITCOM-1276: Notebook to find pairs of stars for star-tracker tests with slew and settle¶

This notebook will produce a list of pairs of stars brighter than Vmag_max, at a given UTC t0, separated by an angular range between sep_lo and sep_hi, and higher than elevation elevation_min.

This is to produce these positions manually. This notebook will be followed up by script that produces a file with the positions and write a json BLOCK for the corresponding observations.

In [ ]:
import astropy.units as u
from astropy.coordinates import AltAz, EarthLocation, SkyCoord
from astropy.time import Time
from astroquery.vizier import Vizier
import pandas as pd

In [ ]:
# Define some configuration variables
Vmag_max = 5 # maximum magnitude of the stars in the list
t0 = '2024-3-8 01:00:00' #in UTC, ideally the central time at which observations will take place
sep_lo = 3.3 #in degrees, minimum separation between stars
sep_hi = 3.7 #in degrees, maximum separation between stars
elevation_min = 45. #  minimum elevation for stars in the pair, NOTE that some might go below
                    # this elevation threshold as the sky rotates during the BLOCK

In [ ]:
# select the site
rubin_site = EarthLocation.of_site('Rubin Observatory') ## equivalent to 'LSST 8.4m' in astropy

In [ ]:
Vizier.ROW_LIMIT=-1
catalogs = Vizier.get_catalogs('V/50') #this the Yale Bright Star Catalog
table = catalogs[0] ## 0 corresponds to catalog in astropy Table format, [1] corresponds to notes

In [ ]:
# select brightest stars
selection_vmag = (table['Vmag'] < Vmag_max)
print(len(table[selection_vmag]))

In [ ]:
ra = table[selection_vmag]['RAJ2000']
dec = table[selection_vmag]['DEJ2000']
stars_radec = SkyCoord(ra,dec,unit=(u.hourangle,u.deg),frame='fk5') #YBSC is in FK5 ref frame

In [ ]:
time = Time(t0) #in UTC
stars_altaz = stars_radec.transform_to(AltAz(obstime=time,location=rubin_site))

In [ ]:
# select stars that are high enough
selection_alt = (stars_altaz.alt > elevation_min*u.deg)
print("Number of stars above elevation threshold:",len(stars_altaz.alt[selection_alt]))

In [ ]:
# print the list
# note that currently the pairs will be repeated as the loop goes through the whole list
# (should be easily fixable)
tmp_list = []
for i,refstar in enumerate(stars_altaz[selection_alt]):
    sep = refstar.separation(stars_altaz[selection_alt])
    selection_sep = (sep > sep_lo*u.deg ) & (sep < sep_hi*u.deg)
    if len(sep[selection_sep]) > 0:
        for j in range(len(sep[selection_sep])):
            tmp_list.append([refstar.transform_to('icrs').ra.deg,
                  refstar.transform_to('icrs').dec.deg,
                  stars_altaz[selection_alt][selection_sep][j].transform_to('icrs').ra.deg,
                  stars_altaz[selection_alt][selection_sep][j].transform_to('icrs').dec.deg,
                  sep[selection_sep][j].deg])
pair_table = pd.DataFrame(tmp_list,columns=['RA star 1 (deg)','Dec star 1 (deg)', 
                                   'RA star 2 (deg)','Dec star 2 (deg)',
                                   'Separation (deg)'])
print(pair_table)


### Visibility of an object
The following cells are a simple set of instructions to find out whether a given position is visible or not

In [ ]:
# check visibility of a star
# HD 37128 RA=84.0533889 DE=-01.2019191
ra = 84.0533889 
dec = -01.2019191
time = Time('2024-3-9 02:14:02') #in UTC

In [ ]:
stars_radec = SkyCoord(ra,dec,unit=(u.deg,u.deg)) #YBSC is in FK5 ref frame
stars_altaz = stars_radec.transform_to(AltAz(obstime=time,location=rubin_site))

In [ ]:
print(stars_altaz)